In [1]:
import geolib.geohash
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

In [2]:
df_original = pd.read_csv('../examples/Vehicles Dataset/data/vehicles_preapred.zip').sort_values(by=["tid", "t"])# precision=5, 50 movelet, DTW
df_original["c1"] = df_original.c1/100000
df_original["c2"] = df_original.c2/100000

df = df_original[["tid", "class", "c1", "c2", "t"]].copy()

df.head()

,tid,class,c1,c2,t
0,30901,B,42.077160,4.738411,0
1,30901,B,42.077246,4.739088,30
2,30901,B,42.077259,4.739096,60
3,30901,B,42.077369,4.739158,90
4,30901,B,42.077635,4.739343,120


In [3]:
tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

spatioTemporalCols = ["c1", "c2", "t"]
n_jobs = 24
verbose = False

In [4]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def compute_measures(test, pred):
    return (accuracy_score(test, pred), precision_score(test, pred, average="micro"), f1_score(test, pred, average="micro"), recall_score(test, pred, average="micro"))


In [5]:
def myconcat_np(big, small):
    if len(big[0]) < len(small[0]):
        return myconcat_np(small, big)

    resized_small = np.empty((len(small), len(big[0])))

    resized_small[:] = np.nan

    big_offset = int(len(big[0])/3)
    small_offset = int(len(small[0])/3)

    for i, row in enumerate(small):
        for k in range(3):
            for j in range(small_offset):
                resized_small[i, k*big_offset+j] = row[k*small_offset+j]

    return np.concatenate((big, resized_small))

In [6]:
from sklearn.feature_selection import mutual_info_classif
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from cri98tj.selectors.selector_utils import select_and_pivot


def my_transform(self, X):
    self._checkFormat(X)

    df = pd.DataFrame(X.copy(), columns=["tid", "class"] + self.spatioTemporalColumns + ["partId"])
    n_class = len(df["class"].unique())

    if self.n_movelets is None:
        self.n_movelets = len(df.partId.unique())  # upper bound
    elif self.n_movelets < 1:
        self.n_movelets = round(self.n_movelets * len(df.partId.unique()))


    movelets_to_test1 = select_and_pivot(df, self.n_movelets/n_class, self.normalizer).values[:, 1:]

    #df.partId = df.partId.apply(lambda x: x[:x.index("_")-1]+x[x.index("_"):])
    new_gh = []
    prec="|"
    prec_tid= -1
    c=-1
    for row in df.values:
        gh = row[-1]
        gh = gh[:gh.index("_")-1]
        if gh != prec or row[0] != prec_tid:
            prec_tid = row[0]
            prec = gh
            c+=1
        new_gh.append(gh+"_"+str(c))

    df.partId = new_gh

    movelets_to_test2 = select_and_pivot(df, self.n_movelets/n_class, self.normalizer).values[:, 1:]

    movelets_to_test = myconcat_np(movelets_to_test1, movelets_to_test2)

    df.partId = df.tid

    if self.n_trajectories is None:
        self.n_trajectories = len(df.partId.unique())  # upper bound
    elif self.n_trajectories < 1:
        self.n_trajectories = round(self.n_trajectories * len(df.partId.unique()))
    trajectories_for_orderline_df = select_and_pivot(df, self.n_trajectories/n_class, self.normalizer)
    trajectories_for_orderline = trajectories_for_orderline_df.drop(columns=["class"]).values
    y_trajectories_for_orderline = trajectories_for_orderline_df[["class"]].values

    scores = []

    if self.verbose: print(F"Computing scores")

    dist_matrix = np.zeros((len(trajectories_for_orderline), len(movelets_to_test)))

    executor = ProcessPoolExecutor(max_workers=self.n_jobs)
    processes = []
    for movelet in tqdm(movelets_to_test, disable=not self.verbose, position=0, leave=True):
        processes.append(executor.submit(self._computeDist, trajectories_for_orderline, movelet))

    for i, process in enumerate(tqdm(processes)):
        res = process.result()
        for j, el in enumerate(res):
            dist_matrix[j][i] = el

    movelets = []
    for i, (score, movelet) in enumerate(sorted(zip(scores, movelets_to_test), key=lambda x: x[0], reverse=True)):
        if i >= self.top_k: break

        if self.verbose: print(F"{i}.\t score={score}")

    mutualInfo = mutual_info_classif(dist_matrix, y_trajectories_for_orderline, n_neighbors=self.n_neighbors,
                                     random_state=self.random_state)

    for i, (score, mov) in enumerate(sorted(zip(mutualInfo, movelets_to_test), key=lambda x: x[0], reverse=True)):
        if i >= self.top_k: break
        movelets.append(mov)
        if self.verbose: print(F"{i}.\t score={score}")

    return movelets  # list of list

In [7]:
%%time

from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

precision = 7

res = []
n_mov_rig = []
time = []
for i in tqdm(range(2, 20)):
    df = df_original[["tid", "class", "c1", "c2", "t"]].copy()
    
    res.append((.0, .0, .0, .0, .0))
    n_mov_rig.append(0)
    time.append(.0)

    for _ in range(3):
        normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
        distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)
        partitioner = Geohash_partitioner(precision=precision, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)

        start = datetime.now()
        part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
        selector = RandomInformationGain_selector(top_k=int(1.4**i), bestFittingMeasure=InterpolatedRootDistanceBestFitting, movelets_per_class=300, trajectories_for_orderline=100, n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
        shapelets = my_transform(selector, part)
        _, dist_np = distancer.fit_transform((df.values, shapelets))
        stop = start - datetime.now()

        n_mov_rig[i-2] += (dist_np.shape[1])

        clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=5000)

        dist_np_df = pd.DataFrame(dist_np)
        X = dist_np_df.drop(columns=[0]).values
        y = dist_np_df[0].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        res[i-2] = tuple(a+b for a, b in zip(compute_measures(y_test, y_pred), res[i-2]))
        time[i-2] += stop.total_seconds()*1000 #millisecondi

    res[i-2] = list(map(lambda x: x/3, res[i-2]))
    n_mov_rig[i-2] /= 3
    time[i-2] /= 3

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

...

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

CPU times: user 22min 12s, sys: 2min 41s, total: 24min 53s
Wall time: 2h 11min 48s


In [8]:
df_res_rig = pd.DataFrame(res, columns=["acc", "prec", "f1", "recall"])

df_res_rig["t"] = time

df_res_rig.t *= -1

df_res_rig["n"] = n_mov_rig

df_res_rig.to_csv(f"Test n_movelet vehicle RIG {precision-1}-{precision}.csv", index=None)

In [9]:
# da qui elimina

In [10]:
%%time

from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

precision = 6

res = []
n_mov_rig = []
time = []
for i in tqdm(range(2, 20)):
    df = df_original[["tid", "class", "c1", "c2", "t"]].copy()
    
    res.append((.0, .0, .0, .0, .0))
    n_mov_rig.append(0)
    time.append(.0)

    for _ in range(3):
        normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
        distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)
        partitioner = Geohash_partitioner(precision=precision, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)

        start = datetime.now()
        part = partitioner.fit_transform(df[df.tid.isin(tid_train)].values)
        selector = RandomInformationGain_selector(top_k=int(1.4**i), bestFittingMeasure=InterpolatedRootDistanceBestFitting, movelets_per_class=300, trajectories_for_orderline=100, n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, normalizer=normalizer, verbose=verbose)
        shapelets = my_transform(selector, part)
        _, dist_np = distancer.fit_transform((df.values, shapelets))
        stop = start - datetime.now()

        n_mov_rig[i-2] += (dist_np.shape[1])

        clf = RandomForestClassifier(max_depth=3, random_state=3, n_jobs=n_jobs, n_estimators=5000)

        dist_np_df = pd.DataFrame(dist_np)
        X = dist_np_df.drop(columns=[0]).values
        y = dist_np_df[0].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        res[i-2] = tuple(a+b for a, b in zip(compute_measures(y_test, y_pred), res[i-2]))
        time[i-2] += stop.total_seconds()*1000 #millisecondi

    res[i-2] = list(map(lambda x: x/3, res[i-2]))
    n_mov_rig[i-2] /= 3
    time[i-2] /= 3

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

...

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

CPU times: user 22min 37s, sys: 2min 40s, total: 25min 17s
Wall time: 7h 22min 5s


In [11]:
df_res_rig = pd.DataFrame(res, columns=["acc", "prec", "f1", "recall"])

df_res_rig["t"] = time

df_res_rig.t *= -1

df_res_rig["n"] = n_mov_rig

df_res_rig.to_csv(f"Test n_movelet vehicle RIG {precision-1}-{precision}.csv", index=None)